In [1]:
import spacy
import json
import os
from nltk.corpus import stopwords, words
import pandas as pd
import gensim
import gensim.corpora as corpora
import re
import contractions
import string

In [396]:
files = []
subdir = tuple(os.walk("../MergedData/merged_movie_jsons/1997"))
for file_name in tuple(subdir)[0][-1]:
    with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
        files.append(json.load(f))

In [397]:
corpus = []
my_pos = set(["ADJ"])
nlp = spacy.load("en_core_web_lg") #python -m spacy download en_core_web_lg

# [print(file["title"]) for file in files]
for file in files:
    if file["title"] != "Titanic":
         continue
    content = [review["content"] for review in file["reviews"]]
    for review in content:
        s = nlp(review)
        clean = set()
        for word in s:
            # if word.pos_ in my_pos:
                clean.add(str(word.lemma_))
        corpus.append(" ".join(clean))
    break

In [398]:
# corpus
# remove non english words again
# remove numbers
# remove words shorted than 3 letters
# remove punctuations
# print(type(stopwords.words("english")))

In [399]:
with open("../stopwords.json", "r") as f:
    common = json.load(f)
    common.extend(stopwords.words("english"))
    common = set(common)


eng_words = set(words.words())

def clean(text: str):
    
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # expand contraction
    text = contractions.fix(text)
    # remove links
    text = re.sub(r"(ftp|http[s]?)://\S+", "", text)
    # remove punctuations
    sub_txt = r"[" + re.escape(string.punctuation) + r"]"
    text = re.sub(sub_txt, "", text)

    text = re.sub("[^a-zA-Z0-9 ]+", " ", text)

    # remove emails
    text = re.sub("[\w\.-]+@[\w\.-]+\.\w+", " ", text)
    
    text = " ".join([word for word in text.split() if word not in common and word in eng_words and len(word) > 2])

     # remove multispaces
    text = re.sub('\s+', " ", text)

    # lemmatizer.lemmatize()
    return text.strip()

In [400]:
proc_data = [clean(movie_revs).split() for movie_revs in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [401]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict,  normalize=True, dictionary=input_dict, slope=2.5, smartirs="Lfc")
vector = model[input_corpus]

In [402]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f
# d

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gensim/models/tfidfmodel.py:180: RuntimeWarning: Mean of empty slice.
  return (1 + np.log2(tf)) / (1 + np.log2(tf.mean(axis=0)))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [403]:
a = {x.lemma_:v for k, v in d.items() for x in nlp(k)if x.pos_ in my_pos}

In [404]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

In [405]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df["TF-IDF"].between(0.8 * df["TF-IDF"].median(), 1.05 * df["TF-IDF"].median(), inclusive=True))
    # pass

In [406]:
# kmeans = KMeans(n_clusters=100, init="k-means++", n_init=10000)
# vect = TfidfVectorizer(stop_words="english")
# features = vect.fit_transform([ " ".join(data) for data in proc_data])
# kmeans.fit(features)

In [407]:
# df2 = pd.DataFrame(corpus)
# df2["clusters"] = kmeans.labels_
# df2

In [408]:
# print("Cluster centrojds: \n")
# order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
# terms = vect.get_feature_names_out()
# for i in range (100):
#     print("cluster %d:" % i)
#     for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
#         print(' %s ' % terms [j])

In [416]:
df = pd.DataFrame(a.values(), index=a.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df["TF-IDF"])
#     pass

boor                   0.889
vacuous                0.678
iconic                 0.655
infamy                 0.644
cathartic              0.641
sentimental            0.613
lovable                0.605
fright                 0.577
decorative             0.574
necessary              0.573
handy                  0.572
unspectacular          0.570
lateral                0.564
unqualified            0.553
monumental             0.553
irrelevant             0.547
pro                    0.541
undisputable           0.533
numb                   0.533
insufficient           0.531
irrational             0.530
cultish                0.529
enchanting             0.513
fanatic                0.510
versatile              0.497
fragmentary            0.489
disgusting             0.488
defective              0.485
asleep                 0.482
unbalanced             0.480
provincial             0.478
caribbean              0.478
fanciful               0.471
expensive              0.464
basilar       

In [415]:


med = df["TF-IDF"].median()
mean = df["TF-IDF"].mean()
vibes = set()
# mean = df["TF-IDF"].quantile(.9)
# a = {k: v for k, v in sorted(a.items(), key=lambda item: item[1], reverse=True)}
for k, v in a.items():
    if abs(med - v) <= 0.025:
        vibes.add(k)
        print(k)
# vibes
# print(file["title"])

epic
fateful
meticulous
rich
cheap
worthy
historical
superior
wonderful
corset
outrageous
atlantic
alive
cozy
dead
dramatic
pleasing
grumpy
mental
silent
vain
stunning
least
natural
unmatched
dynamic
enormous
thin
tremendous
unconvincing
fair
suspenseful
abusive
arrogant
innocent
lovely
wide
accurate
stupendous
old
behold
fantastical
loathsome
loose
cool
ridiculous
theatrical
corny
cheesy
respective
substantial
tough
adventurous
mammoth
concerned
mysterious
erotic
familiar
predictable
unrealistic
brittle
characteristic
loveless
anecdotal
commercial
impervious
vulnerable
pompous
genuine
remote
sensational
egotistical
tired
handsome
oily
constant
penniless
mere
societal
favorable
unlikeable
upset
sublime
alien
fabulous
gentle
conscientious
logical
modest
mythical
hairless
vapid
admirable
initial
rough
stereotypical
superlative
sympathetic
abrasive
beh
infatuate
angry
eerie
stellar
impeccable
arguable
remarkable
eponymous
exemplary
pensive
unusual
creaky
costly
honorable
corrupt
prescient

In [411]:
# cnt = {}
# for sent in corpus:
#     sent = clean(sent)
#     for word in set(sent.split()):
#         cnt[word] = cnt.get(word, 0) + 1

# cnt = {k: v for k, v in sorted(cnt.items(), key=lambda item: item[1], reverse=True)}

# for c in cnt:
#     print(c, cnt[c])